In [138]:
import geopandas as gpd
import pandas as pd
import folium
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

healthcare_df = pd.read_csv('data/healthcare_coverage.csv')
life_df = pd.read_csv('data/life-expectancy.csv')
map = gpd.read_file('data/tl_2020_us_state.zip')

In [139]:
df_long_healthcare = healthcare_df.melt(id_vars=['Year'], var_name='Country', value_name='HealthcareCoverage')
df_long_healthcare['Country'] = df_long_healthcare['Country'].str.split(':').str[0].str.strip()
print(df_long_healthcare.head())

   Year Country HealthcareCoverage
0  1980     AUS                5.8
1  1981     AUS                5.8
2  1982     AUS                6.1
3  1983     AUS                6.0
4  1984     AUS                6.0


In [140]:
df_long_life = life_df.melt(id_vars=['Year'], var_name='Country', value_name='LifeExpectancy')
df_long_life['Country'] = df_long_healthcare['Country'].str.split(':').str[0].str.strip()
print(df_long_life.head())

   Year Country LifeExpectancy
0  1980     AUS               
1  1981     AUS           74.8
2  1982     AUS           74.6
3  1983     AUS           75.4
4  1984     AUS           75.7


In [141]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import pycountry

app = dash.Dash(__name__)

# Example conversion of data columns to numeric, handling missing data
df_long_healthcare['HealthcareCoverage'] = pd.to_numeric(df_long_healthcare['HealthcareCoverage'], errors='coerce')
df_long_life['LifeExpectancy'] = pd.to_numeric(df_long_life['LifeExpectancy'], errors='coerce')

# Calculate global y-axis limits for each topic
healthcare_min = df_long_healthcare['HealthcareCoverage'].min()
healthcare_max = df_long_healthcare['HealthcareCoverage'].max()
life_expectancy_min = df_long_life['LifeExpectancy'].min()
life_expectancy_max = df_long_life['LifeExpectancy'].max()

country_name_mapping = {
    'AUS': 'Australia',
    'CAN': 'Canada',
    'FRA': 'France',
    'GER': 'Germany',
    'JPN': 'Japan',
    'KOR': 'Korea, Republic of',
    'NETH': 'Netherlands',
    'NZ': 'New Zealand',
    'NOR': 'Norway',
    'SWE': 'Sweden',
    'SWIZ': 'Switzerland',
    'UK': 'United Kingdom',
    'US': 'United States'
}

# Replace unrecognized country names with their corresponding ISO 3166-1 alpha-2 codes
df_long_healthcare['Country'] = df_long_healthcare['Country'].replace(country_name_mapping)
df_long_life['Country'] = df_long_life['Country'].replace(country_name_mapping)

country_options = []
for country in df_long_healthcare['Country'].unique():
    country_obj = pycountry.countries.get(name=country)
    if country_obj:
        country_options.append({'label': f"{country} ({country_obj.alpha_2})", 'value': country})
    else:
        print(f"Country not found: {country}")
app.layout = html.Div([
    html.H1("Global Health Data Dashboard"),
    dcc.Dropdown(
        id='topic-dropdown',
        options=[
            {'label': 'Healthcare Coverage', 'value': 'HealthcareCoverage'},
            {'label': 'Life Expectancy', 'value': 'LifeExpectancy'}
        ],
        value='HealthcareCoverage',
        clearable=False
    ),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': 'All Countries', 'value': 'All'}] +
                [{'label': f"{country} ({pycountry.countries.get(name=country).alpha_2})", 'value': country} for country in df_long_healthcare['Country'].unique()],
        value='All',
        clearable=False
    ),
    dcc.Dropdown(
        id='scale-dropdown',
        options=[
            {'label': 'Linear', 'value': 'linear'},
            {'label': 'Logarithmic', 'value': 'log'}
        ],
        value='linear',
        clearable=False
    ),
    dcc.Graph(id='global-health-graph')
])

def country_to_flag(country):
    """ Convert a country name to a flag emoji. """
    try:
        country_obj = pycountry.countries.get(name=country)
        if country_obj:
            return ''.join(chr(0x1F1E6 + ord(c) - ord('A')) for c in country_obj.alpha_2)
        else:
            return ''
    except Exception as e:
        print(f"Error converting country to flag for {country}: {str(e)}")
        return ''

@app.callback(
    Output('global-health-graph', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('topic-dropdown', 'value'),
     Input('scale-dropdown', 'value'),
     Input('global-health-graph', 'hoverData')]
)
def update_figure(selected_country, selected_topic, selected_scale, hoverData):
    if selected_topic == 'LifeExpectancy':
        data_df = df_long_life
        y_axis_title = 'LifeExpectancy'
    else:
        data_df = df_long_healthcare
        y_axis_title = 'HealthcareCoverage'

    fig = go.Figure()

    if selected_country == 'All':
        for i, country in enumerate(data_df['Country'].unique()):
            df_country = data_df[data_df['Country'] == country]
            is_hovered = hoverData and i == hoverData['points'][0]['curveNumber']
            opacity, line_width = (1.0, 6) if is_hovered else (0.3, 2)
            fig.add_trace(go.Scatter(
                x=df_country['Year'],
                y=df_country[y_axis_title],
                mode='lines+markers',
                name=country,
                line=dict(width=line_width),
                opacity=opacity,
                hoverinfo='all',
                hovertemplate=f"{country_to_flag(country)} {country}<br>Year: %{{x}}"
            ))
    else:
        df_country = data_df[data_df['Country'] == selected_country]
        fig.add_trace(go.Scatter(
            x=df_country['Year'],
            y=df_country[y_axis_title],
            mode='lines+markers',
            name=selected_country,
            line=dict(width=2),
            opacity=1.0,
            hoverinfo='all',
            hovertemplate=f"{country_to_flag(selected_country)} {selected_country}<br>Year: %{{x}}"
        ))

    fig.update_layout(
        title=f'{y_axis_title} Trends',
        xaxis_title='Year',
        yaxis=dict(
            title=y_axis_title,
            type=selected_scale,
            range=[data_df[y_axis_title].min(), data_df[y_axis_title].max()] if selected_scale == 'linear' else [None, None]
        ),
        hovermode='closest'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)